In [1]:
from comet_ml import Experiment

In [2]:
import IPython.display as ipd
import numpy as np
import pandas as pd
import librosa
import librosa.display
import os
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, '/Users/jrmylee/Documents/Development/projects/mir/projects/sonata')
import preprocess

from scipy.io import wavfile as wav
from sklearn import metrics 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.regularizers import l1, l2, l1_l2

experiment = Experiment(api_key="HUSGKDDKrFXWOdBLDY3GAhIXD",
                        project_name="general", workspace="jrmyleecape")

In /Users/jrmylee/anaconda3/envs/msmd/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /Users/jrmylee/anaconda3/envs/msmd/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /Users/jrmylee/anaconda3/envs/msmd/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
Using TensorFlow backend.
/Users/jrmylee/anaconda3/envs/msmd/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jrmylee/anaconda3/envs/msmd/

In [29]:
datasets = {
    "isophonics-beetles" : {
        "mp3": "/Users/jrmylee/Documents/Development/projects/mir/datasets/isophonics/beetles_albums",
        "labels": "/Users/jrmylee/Documents/Development/projects/mir/datasets/isophonics/beetles_annotations"
    },
    "isophonics-king" : {
        "mp3": "/Users/jrmylee/Documents/Development/projects/mir/datasets/isophonics/carol_king_albums",
        "labels": "/Users/jrmylee/Documents/Development/projects/mir/datasets/isophonics/carol_king_annotations"
    }
}

def extract_features(audio):
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_processed = np.mean(mfccs.T,axis=0)
    return mfccs_processed

sample_rate = 22050

p = preprocess.Preprocess()

king_albums = p.get_mp3(datasets['isophonics-king']['mp3'])
king_labels = p.get_labels(datasets['isophonics-king']['labels'])
beetles_albums = p.get_mp3(datasets['isophonics-beetles']['mp3'])
beetles_labels = p.get_labels(datasets['isophonics-beetles']['labels'])

data = [
    (king_albums, king_labels),
    (beetles_albums, beetles_labels)
]

In [22]:
import re

def filename_to_title(filename):
    name = re.sub(r'\([^)]*\)', '', filename)
    new_name = ""
    for character in name:
        if character == '.':
            break
        if character.isalnum() and not character.isnumeric():
            new_name += character
    return new_name

def path_to_album(path):
    return os.path.basename(os.path.normpath(path))

def generate_song_labels(label_album_path):
    song_label_dict = {}
    file_labels = labels_dict[label_album_path]
    for file in file_labels:
        if not file['filename'].endswith('.lab'):
            continue
        song_label_dict[file['title']] = []
        with open(os.path.join(label_album_path, file['filename'])) as fp:
            line = fp.readline()
            while line:
                tokens = line.split(' ')
                if len(tokens) == 1: tokens = line.split('\t')
                onset = int(float(tokens[0]))
                offset = int(float(tokens[1]))
                chord = tokens[2][:len(tokens[2]) - 1]
                song_label_dict[file['title']].append((onset, offset, chord))
                line = fp.readline()
    return song_label_dict

In [24]:
def generate_features(albums_dict, album_label_dict):
    features = []
    counter = 0
    for album in albums_dict:
        album_title = path_to_album(album)
        for song in albums_dict[album]:
            counter += 1
            song_path = os.path.join(album, song["filename"])
            song_title = filename_to_title(song["filename"])
            print(str(counter) +"th song: " + song_title)
            data, sr = librosa.load(song_path)
            if album_title in album_label_dict:
                if song_title in album_label_dict[album_title]:
                    for intervals in album_label_dict[album_title][song_title]:
                        start, end, chord = intervals[0], intervals[1], intervals[2]
                        if end > start:
                            start_index = librosa.time_to_samples(start)
                            end_index = librosa.time_to_samples(end)
                            audio_slice = data[int(start_index):int(end_index)]
                            if len(audio_slice) == 0:
                                continue
                            mfccs = librosa.feature.mfcc(y=audio_slice, sr=sample_rate, n_mfcc=40)
                            mfccs_processed = np.mean(mfccs.T,axis=0)
                            features.append([mfccs_processed,  chord])
    return features

In [31]:
features = []
for d in data:
    album_label_dict = {}
    albums_dict = d[0]
    labels_dict = d[1]
    for label_path in labels_dict:
        song_label_dict = generate_song_labels(label_path)
        album_title = path_to_album(label_path)
        album_label_dict[album_title] = song_label_dict
    f = generate_features(albums_dict, album_label_dict)
    features += f
    

1th song: SmackwaterJack


/Users/jrmylee/anaconda3/envs/msmd/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


2th song: HomeAgain


/Users/jrmylee/anaconda3/envs/msmd/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


3th song: ANaturalWoman


...

/Users/jrmylee/anaconda3/envs/msmd/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


253th song: PaperbackWriter


/Users/jrmylee/anaconda3/envs/msmd/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


254th song: AndYourBirdCanSing


/Users/jrmylee/anaconda3/envs/msmd/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [32]:
featuresdf = pd.DataFrame(features, columns=['feature','chord_label'])
featuresdf.head()
len(features)

8170

In [33]:
X = np.array(featuresdf.feature.tolist())
Y = np.array(featuresdf.chord_label.tolist())

le = LabelEncoder()
yy = to_categorical(le.fit_transform(Y))

In [40]:
# Split Dataset into training and test sets
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 127)

In [41]:
#Feedforward NN 
num_labels = yy.shape[1]
filter_size = 2
def build_model_graph(input_shape=(40,)):
    model = Sequential()
    model.add(Dense(256, input_shape=input_shape, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4),
        bias_regularizer=l2(1e-4),
        activity_regularizer=l2(1e-5)))
    model.add(Activation('relu'))
    model.add(Dense(256, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4),
        bias_regularizer=l2(1e-4),
        activity_regularizer=l2(1e-5)))
    model.add(Activation('relu'))
    model.add(Dense(num_labels, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4),
        bias_regularizer=l2(1e-4),
        activity_regularizer=l2(1e-5)))
    model.add(Activation('softmax'))
    # Compile the model
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model
model = build_model_graph()

In [42]:
# Display model architecture summary 
model.summary()
# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_4 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 287)               73759     
_________________________________________________________________
activation_6 (Activation)    (None, 287)               0         
Total params: 150,047
Trainable params: 150,047
Non-trainable params: 0
_________________________________________________________________


In [43]:
print("Pre-training accuracy: %.4f%%" % accuracy)

Pre-training accuracy: 0.3672%


In [44]:
num_epochs = 100
num_batch_size = 32
model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), verbose=1)

COMET WARNING: Set model graph by 'keras' ignored; already called. Future attempts are silently ignored.


Train on 6536 samples, validate on 1634 samples
Epoch 1/100
6536/6536 [==============================] - 1s 160us/step - loss: 10.3525 - acc: 0.1718 - val_loss: 5.3077 - val_acc: 0.2258
Epoch 2/100
6536/6536 [==============================] - 1s 138us/step - loss: 4.4862 - acc: 0.2742 - val_loss: 4.1916 - val_acc: 0.2687
Epoch 3/100
6536/6536 [==============================] - 1s 141us/step - loss: 3.6828 - acc: 0.3297 - val_loss: 3.8289 - val_acc: 0.3121
Epoch 4/100
6536/6536 [==============================] - 1s 140us/step - loss: 3.3047 - acc: 0.3738 - val_loss: 3.6308 - val_acc: 0.3109
Epoch 5/100
6536/6536 [==============================] - 1s 138us/step - loss: 3.0567 - acc: 0.4032 - val_loss: 3.4974 - val_acc: 0.3446
Epoch 6/100
6536/6536 [==============================] - 1s 156us/step - loss: 2.8755 - acc: 0.4487 - val_loss: 3.5002 - val_acc: 0.3305
Epoch 7/100
6536/6536 [==============================] - 1s 137us/step - loss: 2.7484 - acc: 0.4737 - val_loss: 3.4664 - val_acc:

6536/6536 [==============================] - 1s 161us/step - loss: 1.2201 - acc: 0.9267 - val_loss: 3.7912 - val_acc: 0.3911
Epoch 61/100
6536/6536 [==============================] - 1s 152us/step - loss: 1.2738 - acc: 0.9140 - val_loss: 3.7563 - val_acc: 0.3960
Epoch 62/100
6536/6536 [==============================] - 1s 151us/step - loss: 1.3094 - acc: 0.9012 - val_loss: 3.8277 - val_acc: 0.3984
Epoch 63/100
6536/6536 [==============================] - 1s 150us/step - loss: 1.2434 - acc: 0.9207 - val_loss: 3.8475 - val_acc: 0.4021
Epoch 64/100
6536/6536 [==============================] - 1s 150us/step - loss: 1.2472 - acc: 0.9204 - val_loss: 3.7700 - val_acc: 0.3862
Epoch 65/100
6536/6536 [==============================] - 1s 161us/step - loss: 1.2191 - acc: 0.9215 - val_loss: 3.8958 - val_acc: 0.3905
Epoch 66/100
6536/6536 [==============================] - 1s 163us/step - loss: 1.2639 - acc: 0.9105 - val_loss: 3.7962 - val_acc: 0.3923
Epoch 67/100
6536/6536 [=======================

In [45]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))
score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))

Training Accuracy: 92.93%
Testing Accuracy: 38.98%
